In [1]:
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import ProgressBar
from dask.distributed import Client
import glob
import time

In [2]:
varname = 'msl'
varstr = 'mslp'
n_window = 31
data_dir = '/work_big/users/clima/portal/ERA5/'+varstr+'/'
year_range = [2004, 2023]   # for reference climatology

### File selection

In [3]:
# Define the file pattern
file_pattern = data_dir + "ERA5_"+varstr+"_NH_daily_????.nc"
# Get all file paths matching the pattern
all_files = glob.glob(file_pattern)
# Select and sort files
selected_files = []
for year in range(year_range[0], year_range[1]+1):
    selected_files += [file for file in all_files if str(year) in file]
selected_files = sorted(selected_files)
print(selected_files[:5])
print(sorted(all_files)[:5])

[]
[]


### Load data and compute climatology

In [16]:
# Define postprocessing function
def daily_clim(ds, var_name=varname):
    ds_clim = ds.groupby("time.dayofyear").mean("time")[var_name]
    return ds_clim

# Upload data and compute the daily climatology
data_daily = xr.open_mfdataset(
    selected_files, \
    use_cftime=True, \
    combine='by_coords', \
    chunks={'longitude': -1, 'lat': 10,'time': -1}, \
    )
clim_daily = daily_clim(data_daily).compute()

# Save the daily climatology
clim_daily.to_netcdf(data_dir + 'climatology/ERA5_'+varstr+'_NH_daily_clim_'+str(year_range[0])+'-'+str(year_range[1])+'.nc')

ValueError: Every dimension requires a corresponding 1D coordinate and index for inferring concatenation order but the coordinate 'longitude' has no corresponding index

In [9]:
# Extend and smooth the daily climatology with a n_window running mean
n_days = np.floor(n_window / 2).astype(int)
clim_extended = xr.concat(
    [clim_daily[-n_days:], clim_daily, clim_daily[:n_days]],
    dim="dayofyear"
)
new_dayofyear = np.arange(-n_days + 1, 365 + n_days + 1)
clim_extended = clim_extended.assign_coords(dayofyear=new_dayofyear)
clim_smooth = clim_extended.rolling(dayofyear=n_window, center=True).mean()
clim_smooth = clim_smooth.sel(dayofyear=slice(1, 365))

# Save the smoothed climatology
clim_smooth.to_netcdf(data_dir + 'climatology/ERA5_'+varstr+'_NH_daily_clim_'+str(year_range[0])+'-'+str(year_range[1])+'_sm'+str(n_window)+'d.nc')

### Compute anomalies from climatology

In [10]:
for file in sorted(all_files):
    data = xr.open_dataset(file)
    clim = xr.open_dataset(data_dir + 'climatology/ERA5_'+varstr+'_NH_daily_clim_'+str(year_range[0])+'-'+str(year_range[1])+'_sm'+str(n_window)+'d.nc')
    clim = clim.sel(dayofyear=data['time'].dt.dayofyear).drop_vars("dayofyear")
    anom = data - clim
    anom.to_netcdf(data_dir + file.split('/')[-1].split('.')[0] + '_anom.nc')
    print('File ', file.split('/')[-1].split('.')[0] + '_anom.nc is saved')

File  ERA5_z500_NH_daily_1985_anom.nc is saved
File  ERA5_z500_NH_daily_1986_anom.nc is saved
File  ERA5_z500_NH_daily_1987_anom.nc is saved
File  ERA5_z500_NH_daily_1988_anom.nc is saved
File  ERA5_z500_NH_daily_1989_anom.nc is saved
File  ERA5_z500_NH_daily_1990_anom.nc is saved
File  ERA5_z500_NH_daily_1991_anom.nc is saved
File  ERA5_z500_NH_daily_1992_anom.nc is saved
File  ERA5_z500_NH_daily_1993_anom.nc is saved
File  ERA5_z500_NH_daily_1994_anom.nc is saved
File  ERA5_z500_NH_daily_1995_anom.nc is saved
File  ERA5_z500_NH_daily_1996_anom.nc is saved
File  ERA5_z500_NH_daily_1997_anom.nc is saved
File  ERA5_z500_NH_daily_1998_anom.nc is saved
File  ERA5_z500_NH_daily_1999_anom.nc is saved
File  ERA5_z500_NH_daily_2000_anom.nc is saved
File  ERA5_z500_NH_daily_2001_anom.nc is saved
File  ERA5_z500_NH_daily_2002_anom.nc is saved
File  ERA5_z500_NH_daily_2003_anom.nc is saved
File  ERA5_z500_NH_daily_2004_anom.nc is saved
File  ERA5_z500_NH_daily_2005_anom.nc is saved
File  ERA5_z5